# Answering Business Questions using SQL

I will be using SQL to answer some business questions from the Chinook database, which is a database of a fake music retailer.

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [4]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ('table', 'view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
- Regal - Hip-Hop
- Red Tone - Punk
- Meteor and the Girls - Pop
- Slim Jim Bites - Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

Query to find which genres sell best in the USA

1. Subquery to find all tracks sold in the USA
2. Grouping by genre, count the number of tracks sold by their invoice id, tracks_sold. Then divide this by all tracks sold in the USA for percentage_sold

In [30]:
%%sql

WITH usa_tracks_sold AS (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
)

SELECT
    g.name genre,
    COUNT(uts.invoice_line_id) tracks_sold,
    (ROUND(CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) FROM usa_tracks_sold
    ), 3)*100)||'%' percentage_sold
    
    FROM usa_tracks_sold uts
    INNER JOIN track t on t.track_id = uts.track_id
    INNER JOIN genre g on g.genre_id = t.genre_id
    
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10

Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4%
Alternative & Punk,130,12.4%
Metal,124,11.8%
R&B/Soul,53,5.0%
Blues,36,3.4%
Alternative,35,3.3%
Latin,22,2.1%
Pop,22,2.1%
Hip Hop/Rap,20,1.9%
Jazz,14,1.3%


The list of the potential artists is:
- Regal - Hip-Hop
- Red Tone - Punk
- Meteor and the Girls - Pop
- Slim Jim Bites - Blues

Out of these, Blues sells the most, followed by Pop and Hip-Hop. So I will select Slim Jim Bites, Meteor and the Girls, and Regal to add to the store.

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company

In [37]:
%%sql

WITH total_customer AS (
        SELECT 
            c.customer_id,
            c.support_rep_id,
            SUM(i.total) total
        FROM customer c
        INNER JOIN invoice i on i.customer_id = c.customer_id
        GROUP BY 1,2
)


SELECT
    e.employee_id,
    e.first_name || ' ' || e.last_name employee_name,
    '$' || ROUND(SUM(tc.total), 2) employee_sales_total    
FROM total_customer tc
INNER JOIN employee e on employee_id = tc.support_rep_id

GROUP BY 1

Done.


employee_id,employee_name,employee_sales_total
3,Jane Peacock,$1731.51
4,Margaret Park,$1584.0
5,Steve Johnson,$1393.92


Jane Peacock sells the most, while Steve Johnson sells the least. However, Steve Johnson is still selling ~ 80% of the volume of Jane Peacock, so he is still doing well.

My next task is to analyze the sales data for customers from each different country. I have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, I have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, I will group these customers as "Other" in my analysis.



In [46]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order
USA,13,1040.490000000008,7.942671755725252
Canada,8,535.5900000000034,7.047236842105309
Brazil,5,427.68000000000245,7.011147540983647
France,5,389.0700000000021,7.781400000000042
Germany,4,334.6200000000016,8.161463414634186
Czech Republic,2,273.24000000000103,9.108000000000034
United Kingdom,3,245.52000000000078,8.768571428571457
Portugal,2,185.13000000000022,6.3837931034482835
India,2,183.1500000000002,8.72142857142858
Other,15,1094.9400000000085,7.448571428571486




Based on the data, there may be opportunity in the Czech Republic, the United Kingdom, and in India from their high average order size.

However, the number of customers in each of these countries is relatively low, so we should be cautious of spending too much money on advertising here. It would be better to first run small ad campaigns and collect data on new customers to see if the trends hold.


The Chinook store either allows customers to purchase a whole album, or buy individual songs. They are considering saving money by only purchasing songs that are popular from music labels rather than the whole album, and have tasked me to find the amount of customers who buy whole albums vs individual songs. 

I will write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
- Number of invoices
- Percentage of invoices


In [50]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases make up 18% of purchases. I would recommend against eliminating the purchaes of albumns because you would lose almost 1/5 of your revenue.